# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 

print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [5]:
#Check output
print(messages[:10])

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', '$AMD reveal yourself!', '$AAPL Why the drop? I warren Buffet taking out his position?', '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', '$BAC ok good we&#39;re not dropping in price over the weekend, lol', '$AMAT - Daily Chart, we need to get back to above 50.', '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... if BO, then bingo... what is the odds?']


In [6]:
#number of messages
len(messages)


1548010

In [7]:
#Check output
print(sentiments[:10])

[4, 3, 4, 3, 2, 3, 0, 3, 4, 0]


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [9]:
import gensim
from gensim import parsing  

In [10]:
def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    # Lowercase the twit message
    text = message.lower()
    # Replace URLs with a space in the message
    text = re.sub(r'((http\S+)|(www\S+))', ' ', text) 
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'[$]', ' ', text)
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@\S*', ' ', text)
    # Replace everything not a letter with a space
    ## Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    ## Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    ## Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()
    # You can ignore any word that is not longer than one character
    tokens =  gensim.corpora.textcorpus.remove_short(tokens, minsize=2)
    # Lemmatize words using the WordNetLemmatizer. 
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(token) for token in tokens]
    return tokens

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [11]:
# TODO Implement
tokenized = [preprocess(message) for message in messages]

In [12]:
type(tokenized)

list

In [13]:
#check output
print(tokenized[:5])

[['fitb', 'great', 'buy', 'at', 'ill', 'wait'], ['msft'], ['staanalystalert', 'for', 'tdg', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'is', 'buy'], ['amd', 'heard', 'there’s', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits'], ['amd', 'reveal', 'yourself']]


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [14]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""
# TODO: Implement 
all_text = ' '.join([' '.join(tokens) for tokens in tokenized])
bow = Counter()
for word in all_text.split():
    bow[word] += 1
#check ouput
for word, count in bow.most_common(10):
    print('%s:\t %5d' % (word, count))

the:	 398506
to:	 379357
is:	 284833
for:	 273485
amd:	 263969
on:	 241767
of:	 211291
and:	 208462
in:	 205131
this:	 203377


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [15]:
"""
Set the following variables:
    freqs
    low_cutoff
    hihh_cutoff
    K_most_common
"""
# TODO Implement 
# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
import numpy as np
avg = count/len(tokenized)
freqs = {word:np.log(avg) for word,avg in bow.items()} # use log transformation for manageability's sake 


In [16]:
#check output type
type(freqs)
#freqs.items()

dict

In [17]:
# useful function to print a dictionary sorted by value (largest first by default)- taken from bag_of_word exercise
def print_sorted(d, ascending=False):
    factor = 1 if ascending else -1
    sorted_list = sorted(d.items(), key=lambda v: factor*v[1])
    for i, v in sorted_list:
        print("{}: {:.3f}".format(i, v))
print_sorted(freqs)

the: 12.895
to: 12.846
is: 12.560
for: 12.519
amd: 12.484
on: 12.396
of: 12.261
and: 12.248
in: 12.231
this: 12.223
it: 12.096
at: 11.838
mu: 11.805
amzn: 11.794
will: 11.761
fb: 11.728
up: 11.707
aapl: 11.568
are: 11.530
you: 11.425
be: 11.400
nflx: 11.372
short: 11.368
that: 11.364
buy: 11.239
stock: 11.228
today: 11.228
just: 11.200
what: 11.171
down: 11.145
with: 11.139
not: 11.131
call: 11.122
day: 11.116
from: 11.041
we: 11.040
all: 10.996
if: 10.984
now: 10.964
out: 10.945
more: 10.920
wa: 10.919
report: 10.919
they: 10.914
by: 10.896
market: 10.888
but: 10.887
have: 10.877
earnings: 10.875
estimize: 10.874
go: 10.871
like: 10.857
my: 10.846
analyst: 10.813
trade: 10.807
here: 10.796
share: 10.792
going: 10.778
next: 10.757
so: 10.756
get: 10.751
new: 10.745
a: 10.740
ha: 10.732
nvda: 10.724
see: 10.709
good: 10.695
week: 10.688
volume: 10.685
back: 10.669
reporting: 10.659
no: 10.606
quot: 10.600
can: 10.596
time: 10.581
last: 10.540
spy: 10.538
or: 10.531
tomorrow: 10.519
pric

finra: 7.924
flag: 7.923
sept: 7.923
dow: 7.922
night: 7.922
problem: 7.918
earning: 7.918
talking: 7.917
de: 7.916
late: 7.915
txn: 7.915
smh: 7.914
mrk: 7.913
sub: 7.911
releasing: 7.909
worry: 7.903
called: 7.901
i’ll: 7.897
fang: 7.894
available: 7.893
ya: 7.893
boom: 7.892
statement: 7.891
iwm: 7.890
stanley: 7.890
hoping: 7.889
goldman: 7.888
suck: 7.886
security: 7.886
rtn: 7.885
august: 7.883
mid: 7.883
score: 7.883
wave: 7.879
save: 7.875
story: 7.875
moment: 7.874
elliottwave: 7.871
clear: 7.870
hopefully: 7.870
american: 7.870
b: 7.870
hear: 7.867
volatility: 7.867
crap: 7.867
slow: 7.862
tuesday: 7.859
cc: 7.856
jan: 7.855
fly: 7.853
xlf: 7.850
october: 7.849
during: 7.848
seeing: 7.847
user: 7.843
learn: 7.841
ups: 7.840
thursday: 7.839
feeling: 7.838
conference: 7.838
mmm: 7.837
issue: 7.837
question: 7.835
funny: 7.831
yield: 7.829
isn: 7.829
undervalued: 7.828
article: 7.825
channel: 7.825
gone: 7.825
global: 7.824
chinese: 7.823
bid: 7.821
luv: 7.820
america: 7.820
pla

ryzen: 6.879
vice: 6.879
kmb: 6.878
btd: 6.875
act: 6.872
pushing: 6.871
ftv: 6.871
played: 6.870
table: 6.869
battle: 6.869
count: 6.868
jnpr: 6.867
fuel: 6.866
tough: 6.865
ta: 6.864
press: 6.863
hbi: 6.862
aph: 6.862
claim: 6.862
acb: 6.862
auto: 6.859
gdp: 6.859
owner: 6.858
dhi: 6.858
cme: 6.858
pbr: 6.858
fomo: 6.858
nem: 6.856
bouncing: 6.854
mobile: 6.852
mdlz: 6.852
boeing: 6.851
shorty: 6.849
stifel: 6.846
airline: 6.844
foxa: 6.843
terrible: 6.842
nlsn: 6.841
eem: 6.841
orly: 6.841
pure: 6.838
production: 6.837
totally: 6.837
versus: 6.831
oxy: 6.831
split: 6.829
ticker: 6.828
significantly: 6.827
semiconductor: 6.827
pricing: 6.827
pathetic: 6.827
antm: 6.825
gained: 6.823
bloomberg: 6.823
staying: 6.822
grmn: 6.822
white: 6.821
lod: 6.821
mistake: 6.820
comparing: 6.820
cog: 6.819
bargain: 6.818
regret: 6.818
hsy: 6.817
spending: 6.817
deutsche: 6.817
adi: 6.816
january: 6.816
boring: 6.816
lucky: 6.815
jesus: 6.815
ag: 6.815
print: 6.813
tired: 6.812
screwed: 6.812
schedu

aren’t: 6.295
corner: 6.295
bhc: 6.295
nlst: 6.295
wti: 6.293
concerned: 6.293
xray: 6.293
bright: 6.293
pile: 6.293
extreme: 6.292
officially: 6.290
midterm: 6.290
chain: 6.290
busy: 6.290
mill: 6.290
expired: 6.290
sogo: 6.288
vmc: 6.288
stage: 6.288
cyclical: 6.288
delayed: 6.288
lg: 6.288
worker: 6.286
ltd: 6.286
meant: 6.286
inverted: 6.286
slaughtered: 6.284
idk: 6.284
snp: 6.282
def: 6.282
ivz: 6.282
ing: 6.282
citron: 6.280
ipg: 6.280
casino: 6.279
siri: 6.279
dear: 6.279
hundred: 6.279
child: 6.279
rok: 6.277
nowhere: 6.277
disposition: 6.277
jec: 6.277
speak: 6.275
smoke: 6.275
blog: 6.273
cm: 6.273
iran: 6.273
lyb: 6.271
russia: 6.271
cutting: 6.271
heck: 6.271
ti: 6.271
fee: 6.269
biz: 6.269
rough: 6.269
challenge: 6.269
honest: 6.269
ssc: 6.269
original: 6.267
xec: 6.267
bigly: 6.267
proof: 6.265
sport: 6.263
math: 6.263
anti: 6.263
oke: 6.260
nept: 6.260
ntrs: 6.258
provide: 6.258
among: 6.258
fe: 6.256
continued: 6.256
stable: 6.256
skx: 6.256
analyzer: 6.254
damage: 6.2

everytime: 5.781
adj: 5.781
runner: 5.778
provided: 5.778
canaccord: 5.778
rival: 5.775
kroger: 5.775
bail: 5.775
preview: 5.775
hair: 5.775
instrument: 5.775
figured: 5.775
suggested: 5.771
choppy: 5.771
sir: 5.771
rigged: 5.771
chatter: 5.771
ig: 5.771
causing: 5.768
sun: 5.768
etr: 5.768
harami: 5.768
scandal: 5.768
stayed: 5.768
sucked: 5.768
realistic: 5.768
swinging: 5.768
wll: 5.765
focused: 5.765
eh: 5.765
neg: 5.765
toilet: 5.765
evercore: 5.765
bang: 5.765
senate: 5.765
became: 5.762
downturn: 5.762
manipulating: 5.762
emerging: 5.762
gov: 5.762
edward: 5.762
initiation: 5.762
named: 5.762
kicked: 5.762
slm: 5.762
gogo: 5.762
minor: 5.762
hclp: 5.762
science: 5.759
lynch: 5.759
choose: 5.759
es: 5.759
expand: 5.759
sweeper: 5.759
kicker: 5.759
retard: 5.759
copper: 5.759
stellar: 5.759
blew: 5.759
justify: 5.756
rsg: 5.756
electronic: 5.756
celgene: 5.753
mmc: 5.753
dipped: 5.753
dinner: 5.753
depot: 5.753
cake: 5.753
himself: 5.753
peter: 5.753
bless: 5.753
activist: 5.753
g

...

errnings: 0.000
tuiff: 0.000
‼️‼️: 0.000
mcsteamy: 0.000
😅🤙🏼: 0.000
impedes: 0.000
beardisciplesshorting: 0.000
offwhites: 0.000
😁🤑🤣: 0.000
upside’: 0.000
😉😜😜😜: 0.000
losses‼️‼️: 0.000
transacts: 0.000
fallbelow: 0.000
aapl🍎and: 0.000
diseconomies: 0.000
fall‼️: 0.000
cappuccino: 0.000
ludicrousness: 0.000
🥂for: 0.000
😉‼️: 0.000
barn’s: 0.000
😘❤️: 0.000
ooohyeah: 0.000
antiehr: 0.000
thoo: 0.000
now🤔: 0.000
pleeeeeeeeease: 0.000
acompany: 0.000
silentbullshit: 0.000
tarfiif: 0.000
negativestockratings: 0.000
intelligentforesight: 0.000
selffulfillingprophecies: 0.000
bearraids: 0.000
targt: 0.000
oucha: 0.000
⬆️when: 0.000
😌😌: 0.000
ouck: 0.000
n💖big: 0.000
💋🍎: 0.000
leygo: 0.000
🔽🔻⏬: 0.000
jcrashing: 0.000
😂😂😂🤡: 0.000
tthanks: 0.000
dewm: 0.000
ftws: 0.000
mlcg: 0.000
nnrx: 0.000
psru: 0.000
czni: 0.000
holding🤓: 0.000
stopped’: 0.000
makefordgreatagain💪🏽🇺🇸: 0.000
🍏💖: 0.000
excuberance: 0.000
dilutesp: 0.000
dcbs: 0.000
yyyyyasssssssss: 0.000
baeee: 0.000
belichek: 0.000
innacurrate: 

casion: 0.000
suzzanne: 0.000
✌✌✌✌: 0.000
ralllying: 0.000
azklf: 0.000
brainchip: 0.000
beamer: 0.000
qued: 0.000
poned: 0.000
supend: 0.000
repurchace: 0.000
multiplea: 0.000
coolbrith: 0.000
gahahajajjaaaajrhejejwkwms: 0.000
whywhwhwhwhwuwyw: 0.000
greatz: 0.000
this📰: 0.000
ibmtransformation: 0.000
goingoing: 0.000
calllllllllllll: 0.000
callllllll😆😆🤑🤑🤑🤑🤑🤑🤑🤑🤑: 0.000
honeywell…: 0.000
cryptocurrecy: 0.000
sekey: 0.000
drm: 0.000
wannabes📈😁😄😆😂🤣📈: 0.000
🍜😂: 0.000
shits📈📈: 0.000
yayyyyyyyy😎😎😎😎: 0.000
grandad: 0.000
yeeepee: 0.000
😪🍆👅🍑💦📈: 0.000
magi: 0.000
teamliquid: 0.000
👍🏽📈: 0.000
manipuation: 0.000
raydalio: 0.000
actavis: 0.000
sosososo: 0.000
🙌🏼🍻🔥: 0.000
📈for: 0.000
highlighter: 0.000
undervalueddddddddddddddddd: 0.000
knkws: 0.000
aplaud: 0.000
👏👏👏👍: 0.000
stockpikr: 0.000
🐂get: 0.000
catfshing: 0.000
water😁🎣: 0.000
ktrader: 0.000
synergetic: 0.000
firns: 0.000
“options”: 0.000
bullcarp: 0.000
dipshts: 0.000
took’em: 0.000
💪🏻💰🚀: 0.000
🤪💵😜💵💪💪💪: 0.000
choles: 0.000
xcelent: 0.000


innovat: 0.000
nyv: 0.000
fkwl: 0.000
ftmdf: 0.000
announced？: 0.000
succumbing: 0.000
visibly: 0.000
imperialism: 0.000
abitions: 0.000
kijsen: 0.000
greenwood: 0.000
believeinsu🌞⭐️💫: 0.000
harrah’s: 0.000
cryptominning: 0.000
yolandi: 0.000
visser: 0.000
antwoord: 0.000
thenthis: 0.000
🤞🏽if: 0.000
knowbody: 0.000
bearporn: 0.000
dwai: 0.000
pervaisve: 0.000
rockstars: 0.000
armament: 0.000
⬇️post: 0.000
🙄🤔🤑🤓: 0.000
lol🤦‍♂️😆: 0.000
obamasky: 0.000
😖😲: 0.000
scopic: 0.000
🎃☠👻: 0.000
jihua: 0.000
🤫🤭😂: 0.000
continue🤗: 0.000
ruin😉: 0.000
nexttariff: 0.000
pt—: 0.000
worthers: 0.000
bldps: 0.000
precioous: 0.000
euphemism: 0.000
apostoo: 0.000
gsh: 0.000
carls: 0.000
”until: 0.000
gutter”: 0.000
ktcc: 0.000
dehumidifier: 0.000
felix: 0.000
journey”: 0.000
journeying: 0.000
scenario’s: 0.000
kazakhstan: 0.000
beyondddd: 0.000
ahs😳: 0.000
l’aube: 0.000
pitot: 0.000
inoperative: 0.000
mktt: 0.000
cheap😉: 0.000
saying🤷‍♀️: 0.000
réalist: 0.000
air’s: 0.000
batam: 0.000
fondamenta: 0.000
listé

sala: 0.000
🐥👍🏼: 0.000
market🚀🚀🚀🚀❤️❤️: 0.000
bwhahahahah: 0.000
aherd: 0.000
rufus: 0.000
bought🚀🚀🚀🚀: 0.000
cheapish: 0.000
😋🇺🇸🤗: 0.000
—if: 0.000
‘oh—last: 0.000
share’: 0.000
triplicated: 0.000
💥🌶💥: 0.000
excllent: 0.000
squezzzzzeeeeee: 0.000
pessimissm: 0.000
🚀🚀🎉🚀❤️: 0.000
grin😉s: 0.000
cuffed: 0.000
fanger: 0.000
decli: 0.000
cometo: 0.000
🎉👍🔥🔥🔥: 0.000
here👍🏼: 0.000
bears🤣: 0.000
“slowest: 0.000
theverge: 0.000
anywel: 0.000
googlebook: 0.000
whhooohooo: 0.000
🚶🏿‍♀️🚶🏿‍♀️🚶🏿‍♀️🚶🏿‍♀️🚶🏿‍♀️🚶🏿‍♀️: 0.000
fucklach: 0.000
errrbody: 0.000
“shrinked”: 0.000
long🚀🚀🚀🎈🎉🎉🔥🔥: 0.000
shoykd: 0.000
pooooor: 0.000
memaw: 0.000
spambots: 0.000
facecrypto: 0.000
bakey: 0.000
🏃‍♀️🏃‍♀️: 0.000
checknyour: 0.000
representsconsistent: 0.000
breakingviews: 0.000
acquisitionsin: 0.000
🚶🏿‍♀️🏃‍♀️🏃‍♀️keep: 0.000
bless🙏🏼🌎💰🔥: 0.000
milliseconds—: 0.000
🏄🏾: 0.000
🍻🍻👀👀👀: 0.000
fookerz: 0.000
🙌🏻✔️: 0.000
rari: 0.000
maus😱: 0.000
🐻🔥🔥😏: 0.000
faaaassstttt: 0.000
thanksgiven: 0.000
buyyyyyyyyyyyyyyyyyyyyyyy: 0.000
bulls

moreresearch: 0.000
beststockonthemarket: 0.000
drilll: 0.000
🤠👍🎉: 0.000
“box: 0.000
chocolates”: 0.000
gizzard: 0.000
🤔🤔super: 0.000
baldemort: 0.000
holeechow: 0.000
invigorate: 0.000
retransformation: 0.000
mainpulative: 0.000
casionos: 0.000
level😅: 0.000
whiever: 0.000
bamboozling: 0.000
‘to: 0.000
moon’: 0.000
summ: 0.000
it🙆🏾‍♂️🙆🏾‍♂️🙈🙈: 0.000
‘stellar’: 0.000
ruthlessness: 0.000
hovercraft: 0.000
suppoed: 0.000
pos🤷‍♂️: 0.000
tober: 0.000
inappropriately: 0.000
bullspread: 0.000
chet: 0.000
valuve: 0.000
gutfeld: 0.000
punchable: 0.000
count💁‍♀️🇺🇸: 0.000
caseclosed: 0.000
keepingitsimple: 0.000
gfornit: 0.000
tost: 0.000
yooooolooooo: 0.000
radsellschipstoo: 0.000
whyi: 0.000
😏🤦‍♀️: 0.000
obalup: 0.000
stepdaughter: 0.000
😂😭😂😭😂: 0.000
oceanmedallion: 0.000
drop🙈: 0.000
gatherer: 0.000
grand😂😂😂😂: 0.000
radisbetterthanfred: 0.000
ssfn: 0.000
nubers: 0.000
introductory: 0.000
viaga: 0.000
screen😄: 0.000
👀👀🚀🤑: 0.000
harddd: 0.000
😅🎃: 0.000
ruun: 0.000
⬆️upswung: 0.000
stock⬆️: 0.000

In [18]:
#Check output
#high_cutoff = 12
#K_most_common = [word for word,count in bow.most_common(high_cutoff)]
#K_most_common[:10]
#len(K_most_common)

In [19]:
# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
#if below 12 stocks names might be affected.
high_cutoff = 12

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word,count in bow.most_common(high_cutoff)]

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
#print(K_most_common)
len(filtered_words) 

57836

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [20]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement
# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word:idx for idx,word in enumerate(filtered_words, 1)}
#print(vocab)
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {idx:word for idx,word in enumerate(filtered_words, 1)}
#print(id2vocab)
# tokenized with the words not in `filtered_words` removed.

from tqdm import tqdm_notebook as tqdm
filtered = [[w for w in message if w in vocab] for message in tqdm(tokenized)]#taken from the student hub

HBox(children=(IntProgress(value=0, max=1548010), HTML(value='')))

In [21]:
#check output
print(filtered[:10])

[['fitb', 'great', 'buy', 'ill', 'wait'], ['msft'], ['staanalystalert', 'tdg', 'jefferies', 'maintains', 'with', 'rating', 'hold', 'setting', 'target', 'price', 'usd', 'our', 'own', 'verdict', 'buy'], ['heard', 'there’s', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'stocktwits'], ['reveal', 'yourself'], ['aapl', 'why', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position'], ['ba', 'bear', 'have', 'reason', 'pay', 'more', 'attention'], ['bac', 'ok', 'good', 'we', 're', 'not', 'dropping', 'price', 'over', 'weekend', 'lol'], ['amat', 'daily', 'chart', 'we', 'need', 'get', 'back', 'above'], ['gme', 'drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'month', 'back', 'if', 'bo', 'then', 'bingo', 'what', 'odds']]


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [22]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [23]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1990277410530681

Finally let's convert our tokens into integer ids which we can pass to the network.

In [24]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [25]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


### View Model

In [48]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement
        
              
        

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout = dropout, batch_first = False)
        
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Setup additional layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.lsof = nn.LogSoftmax(dim = 1)
        
        
    def init_hidden(self, batch_size):
            
        """ 
        Initializes hidden state
        Parameters
        ----------
            batch_size : The size of batches.
        Returns
        -------
        hidden_state        
        """
        
        # TODO Implement 
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        
        hidden_state = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                        weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden_state

       
    

    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        ## embeddings and lstm_out
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        # stack up lstm outputs
        lstm_out = lstm_out[-1,:,:]
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        #Softmax function
        logps = self.lsof(out)
        
        # reshape to be batch_size first
        batch_size = nn_input.size(1)    
                       
        # return last sigmoid output and hidden state
        return logps, hidden_state   
  

In [49]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.6212, -1.7127, -1.8007, -1.5458, -1.4121],
        [-1.6259, -1.6958, -1.8471, -1.5471, -1.3893],
        [-1.5403, -1.7768, -1.8048, -1.4039, -1.5781],
        [-1.6088, -1.6753, -1.8072, -1.5509, -1.4420]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [50]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [51]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.8
split_idx = int(len(token_ids)*split_frac)

train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]
     

In [52]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(57837, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (lsof): LogSoftmax()
)

In [54]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 4
batch_size = 1024
learning_rate = 0.001
clip = 5


print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# move model to GPU, if available
if(train_on_gpu):
    model.cuda()

model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(train_features, train_labels, batch_size=batch_size):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
             
                                
        # zero accumulated gradients
        model.zero_grad()
        
        # get the output from the model
        output, hidden = model.forward(text_batch, hidden)
               
        # calculate the loss and perform backpropagation
        loss = criterion(output.squeeze(),labels)
        loss.backward()
        
       
        # clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
                   
        if steps % print_every == 0:
            model.eval()
            
            
            val_losses = []
            accuracy = []
            
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size=batch_size):
            
                
                inputs, labels = inputs.to(device), labels.to(device)
                    
                val_h = model.init_hidden(labels.shape[0])
                for each in val_h:
                    each.to(device)
                        
                output, val_h = model.forward(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)
                val_losses.append(val_loss.item())
                
                ps = torch.exp(output)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())
                    
            
            model.train()
            # TODO Implement: Print metrics
            print('Epoch: {}/{}...'.format(epoch+1, epochs),
                  'Step: {}...'.format(steps),
                  'Loss: {:.6f}...'.format(loss.item()),
                  'Val Loss: {:.6f}'.format(np.mean(val_losses)),
                  'Accuracy: {:.6f}'.format(np.mean(accuracy)))
                                  
            

Starting epoch 1
Epoch: 1/4... Step: 100... Loss: 0.962953... Val Loss: 1.043019 Accuracy: 0.584679
Epoch: 1/4... Step: 200... Loss: 0.960403... Val Loss: 0.922775 Accuracy: 0.642400
Epoch: 1/4... Step: 300... Loss: 0.790872... Val Loss: 0.862542 Accuracy: 0.663973
Epoch: 1/4... Step: 400... Loss: 0.860975... Val Loss: 0.839353 Accuracy: 0.668527
Epoch: 1/4... Step: 500... Loss: 0.663760... Val Loss: 0.791811 Accuracy: 0.697479
Epoch: 1/4... Step: 600... Loss: 0.820661... Val Loss: 0.773120 Accuracy: 0.705094
Epoch: 1/4... Step: 700... Loss: 0.669646... Val Loss: 0.754575 Accuracy: 0.712967
Epoch: 1/4... Step: 800... Loss: 0.796675... Val Loss: 0.745146 Accuracy: 0.715213
Starting epoch 2
Epoch: 2/4... Step: 100... Loss: 0.706630... Val Loss: 0.762908 Accuracy: 0.707606
Epoch: 2/4... Step: 200... Loss: 0.787075... Val Loss: 0.740336 Accuracy: 0.717278
Epoch: 2/4... Step: 300... Loss: 0.674046... Val Loss: 0.743093 Accuracy: 0.711095
Epoch: 2/4... Step: 400... Loss: 0.754235... Val Loss

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [55]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [w for w in tokens if w in vocab]
    # Convert words to ids
    tokens = [vocab[idx] for idx in tokens]
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1,1)))
    # Get the NN output
    hidden = model.init_hidden(1)
      
    logps, _ = model.forward(text_input, hidden)
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps.squeeze())
    
    return pred

In [56]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to('cpu')
predict(text, model, vocab)

tensor([ 0.0001,  0.0198,  0.0030,  0.8951,  0.0820], device='cpu')

# Questions: What is the prediction of the model? What is the uncertainty of the prediction?

**TODO: Answer Question**

The ultimate goal of the current model is to learn patterns that generalize well for unseen data. Comparing predictions carried on the validation dataset to the actual answers allow us to assess the performance of the model. Assuming that the latter fits the data, the only difference between the predicted value and a particular observed response should be a random error with a mean of zero. If this is the case, there is no additional information to gain from the model, rarely the case in the first stages of training the model. In such cases. entropy comes into play, a measure of uncertainty of random variables, which includes the uncertainty of the estimated average response and the uncertainty of the new measure expressed as a probability distribution. This phenomenon is captured by the Loss function which quantifies the gap between predicted value and actual label. In a non-negative value, robustness of the model increases as the Loss function decreases. In above mentioned model, it is observable that in epoch 4 the model has stopped learning: Accuracy starts to decrease, Loss increases, Val Loss hints at the model memorizing data shown in the training stage.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [57]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [58]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [59]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [60]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([ 0.0535,  0.0968,  0.2146,  0.2613,  0.3738], device='cpu'),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.